In [ ]:
!pip install transformers datasets accelerate --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving fikralarFinal.json to fikralarFinal.json


In [ ]:
import json
from datasets import Dataset

# JSON dosyasını açın:
with open("fikralarFinal.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 'icerik' alanını alarak listeye dönüştürün:
texts = [item["icerik"] for item in data if "icerik" in item]

# Dataset oluşturun:
dataset = Dataset.from_dict({"text": texts})

# Train-test bölmesi yapın (%90 train, %10 test)
dataset = dataset.train_test_split(test_size=0.1)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "ytu-ce-cosmos/turkish-gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)




Map:   0%|          | 0/1086 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    eval_strategy="epoch",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none"
)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,1.609700,2.148162
2,1.265700,2.152499
3,1.154100,2.119154
4,0.872000,2.165304
5,0.490300,2.205272
6,0.487800,2.225113
7,0.389800,2.240606
8,0.332000,2.261171
9,0.268300,2.264718
10,0.285000,2.267633


TrainOutput(global_step=1360, training_loss=0.7010477583197986, metrics={'train_runtime': 588.1415, 'train_samples_per_second': 18.465, 'train_steps_per_second': 2.312, 'total_flos': 709407866880000.0, 'train_loss': 0.7010477583197986, 'epoch': 10.0})

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
output = generator(
    "Nasreddin Hoca",
    max_length=150,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.9,
    top_p=0.95
)
print(output[0]["generated_text"])


Device set to use cuda:0


Nasreddin Hoca'nın bir sürü esegi varmış. Mesela ben desem, Kuba'nın en esegi diye anilacak seyler bulamam. Hoca'yla biraz dalga geçmek için, Kuba'ya gitmis bir de bakmislar ki, adeta aliyyül-ahlak sahibi. Kuba'yi anliyamiyorlar. Bir de ne görsün, Kuba'nin bütün esegi altinmis. Hoca, "Ben desem, Kuba'nun en dehsetkar hareketi diye anilacak sey yok" diyor ve devam ediyor: "Bana öyle bir siir yaz ki, bütün Kuba'lilar okumasini yapip sana versinler..." :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :) :);


In [ ]:

model.save_pretrained("fikra-model")
tokenizer.save_pretrained("fikra-model")


('fikra-model/tokenizer_config.json',
 'fikra-model/special_tokens_map.json',
 'fikra-model/vocab.json',
 'fikra-model/merges.txt',
 'fikra-model/added_tokens.json',
 'fikra-model/tokenizer.json')

In [ ]:
!zip -r fikra-model.zip fikra-model


  adding: fikra-model/ (stored 0%)
  adding: fikra-model/special_tokens_map.json (deflated 74%)
  adding: fikra-model/vocab.json (deflated 62%)
  adding: fikra-model/tokenizer.json (deflated 82%)
  adding: fikra-model/config.json (deflated 52%)
  adding: fikra-model/model.safetensors (deflated 7%)
  adding: fikra-model/merges.txt (deflated 60%)
  adding: fikra-model/generation_config.json (deflated 31%)
  adding: fikra-model/tokenizer_config.json (deflated 56%)


In [ ]:
from google.colab import files
files.download("fikra-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>